In [1]:
from typing import Generator, List, Any, Type, Dict, Tuple

import src.systems.request_completion.nodes.nodes as no
from src.systems.request_completion.nodes.nodes import NodeFactory
from src.systems.request_completion.nodes.library import FunctionNode
from src.systems.request_completion.state.context.context import BaseContext
import src.utils.model_llm as llm 

from enum import Enum

from dotenv import load_dotenv
load_dotenv()


True

In [10]:


class WeatherForecaster(no.Node[str]):
    
    @classmethod
    def system_prompt(cls) -> str:
        return """Your goal is to provide the user with a weather forecast. You should use the tool calls available to you to get the information you need."""
    
    @classmethod
    def tools(cls) -> llm.ToolListDefinition:
        return llm.ToolListDefinition([
            llm.ToolDefinition("get_current_weather", "Collects the current weather at the given location", parameters=[llm.Parameter("location", "string", description="The location you would like the current weather", required=True)]),
            llm.ToolDefinition("get_weekly_forecast", "Collects the weekly weather forecast at the given location", parameters=[llm.Parameter("location", "string", description="The location you would like the forecast", required=True)]),
            llm.ToolDefinition("get_long_term_forecast", "Collects the long term weather forecast at the given location", parameters=[llm.Parameter("location", "string", description="The location you would like the forecast", required=True)]),
        ])
    
    def get_node_from_tool_call(self, tool_call_name: str):
        """
        Below you can see two ways one can have a node that is simply calling a function
        A: Use the provided `FunctionNode` class which only needs to be initilized with the
        required callable and keyword arguments
        B: Inheriting from `TerminalNode` and defining the required `invoke` method

        """
        # Using Option A    
        if tool_call_name == "get_current_weather":
            return lambda location: FunctionNode(get_current_weather, location=location)
        
        # Using Option B - The defined node classes can be found at the bottom of the notebook
        if tool_call_name == "get_weekly_forecast":
            return WeeklyForecast
        if tool_call_name == "get_long_term_forecast":
            return LongTermForecast
    
    
    def __init__(self, mess_hist: llm.MessageHistory):
        super().__init__()
        self.message_history = mess_hist.insert_message(llm.SystemMessage(self.system_prompt()), 0)
        self.model = llm.ModelLLM(llm.APIParent.OPENAI, "gpt-4o")
        

    def invoke(self) -> str:
        while True:
            try:
                response = self.model.call_llm(self.message_history, tools=self.tools())
                self.message_history.add_message(response)
            except llm.LLMException as e:
                raise no.ResetException(self, str(e.original_error) )
            
            if response.tool_calls:
                responses = self.call_nodes([NodeFactory(self.get_node_from_tool_call(t_c.name), **t_c.kwargs) for t_c in response.tool_calls])
                
                handled_responses = [self.handle_response(r.data) for r in responses]
                
                for request_id, tool_resp in zip([x.identifier for x in response.tool_calls], handled_responses):
                    self.message_history.add_tool_response(request_id, tool_resp)
            
            if response.content is not None:
                break
        
        return response.content.text
                
    def handle_response(self, output: Any) -> str:
        if isinstance(output, Tuple):
            return str(output)

        if isinstance(output, str):
            return output
        
        if isinstance(output, list):
            return "\n".join(str(item) for item in output)
        
        raise no.FatalException(self, f"Unknown output {output} {output}")
    
        
    @classmethod
    def pretty_name(cls):
        return "Top Level Weather"

In [11]:

# in this cell I am making some mock function calls that you can imagine would be API calls if you were looking for an accurate system
class WeatherType(Enum):
    SUNNY = 'sunny'
    RAINY = 'rainy'
    CLOUDY = 'cloudy'
    SNOWY = 'snowy'
    WINDY = 'windy'
    
class LocationNotFoundException(Exception):
    pass

def get_current_weather(location: str) -> Tuple[float, WeatherType]:
    location_weather_map = {
        "New York": (75.0, WeatherType.SUNNY),
        "Los Angeles": (90.0, WeatherType.CLOUDY),
        "Chicago": (65.0, WeatherType.RAINY),
        "Miami": (85.0, WeatherType.WINDY),
        "Seattle": (55.0, WeatherType.SNOWY),
        "Orlando": (80.0, WeatherType.SUNNY),
        "Denver": (70.0, WeatherType.CLOUDY),
        "San Francisco": (75.0, WeatherType.RAINY),
        "Boston": (65.0, WeatherType.WINDY),
        "Las Vegas": (90.0, WeatherType.SNOWY),
        "Vancouver": (55.0, WeatherType.SUNNY),
        "Toronto": (80.0, WeatherType.CLOUDY),
        "Portland": (55.0, WeatherType.RAINY),
        "Austin": (90.0, WeatherType.WINDY),
        "Houston": (85.0, WeatherType.SNOWY),
        "Springfield": (65.0, WeatherType.SUNNY),
    }
    
    if location not in location_weather_map:
        raise LocationNotFoundException(f"Location {location} not found")
    
    return location_weather_map[location]

def get_weekly_forecast(location: str) -> List[Tuple[float, WeatherType]]:
    location_forecast_map = {
        "New York": [
            (75.0, WeatherType.SUNNY),
            (70.0, WeatherType.CLOUDY),
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
            (55.0, WeatherType.SNOWY),
            (50.0, WeatherType.SUNNY),
            (45.0, WeatherType.CLOUDY),
        ],
        "Los Angeles": [
            (90.0, WeatherType.CLOUDY),
            (85.0, WeatherType.WINDY),
            (80.0, WeatherType.SNOWY),
            (75.0, WeatherType.SUNNY),
            (70.0, WeatherType.CLOUDY),
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
        ],
        "Chicago": [
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
            (55.0, WeatherType.SNOWY),
            (50.0, WeatherType.SUNNY),
            (45.0, WeatherType.CLOUDY),
            (40.0, WeatherType.RAINY),
            (35.0, WeatherType.WINDY),
        ],
        "Miami": [
            (85.0, WeatherType.WINDY),
            (80.0, WeatherType.SNOWY),
            (75.0, WeatherType.SUNNY),
            (70.0, WeatherType.CLOUDY),
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
            (55.0, WeatherType.SNOWY),
        ],
        "Seattle": [
            (55.0, WeatherType.SNOWY),
            (50.0, WeatherType.SUNNY),
            (45.0, WeatherType.CLOUDY),
            (40.0, WeatherType.RAINY),
            (35.0, WeatherType.WINDY),
            (30.0, WeatherType.SNOWY),
            (25.0, WeatherType.SUNNY),
        ],
        "Orlando": [
            (80.0, WeatherType.SUNNY),
            (75.0, WeatherType.CLOUDY),
            (70.0, WeatherType.RAINY),
            (65.0, WeatherType.WINDY),
            (60.0, WeatherType.SNOWY),
            (55.0, WeatherType.SUNNY),
            (50.0, WeatherType.CLOUDY),
        ],
        "Denver": [
            (70.0, WeatherType.CLOUDY),
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
            (55.0, WeatherType.SNOWY),
            (50.0, WeatherType.SUNNY),
            (45.0, WeatherType.CLOUDY),
            (40.0, WeatherType.RAINY),
        ],
        "San Francisco": [
            (75.0, WeatherType.RAINY),
            (70.0, WeatherType.WINDY),
            (65.0, WeatherType.SNOWY),
            (60.0, WeatherType.SUNNY),
            (55.0, WeatherType.CLOUDY),
            (50.0, WeatherType.RAINY),
            (45.0, WeatherType.WINDY),
        ],
        "Boston": [
            (65.0, WeatherType.WINDY),
            (60.0, WeatherType.SNOWY),
            (55.0, WeatherType.SUNNY),
            (50.0, WeatherType.CLOUDY),
            (45.0, WeatherType.RAINY),
            (40.0, WeatherType.WINDY),
            (35.0, WeatherType.SNOWY),
        ],
        "Las Vegas": [
            (90.0, WeatherType.SNOWY),
            (85.0, WeatherType.SUNNY),
            (80.0, WeatherType.CLOUDY),
            (75.0, WeatherType.RAINY),
            (70.0, WeatherType.WINDY),
            (65.0, WeatherType.SNOWY),
            (60.0, WeatherType.SUNNY),
        ],
        "Vancouver": [
            (55.0, WeatherType.SUNNY),
            (50.0, WeatherType.CLOUDY),
            (45.0, WeatherType.RAINY),
            (40.0, WeatherType.WINDY),
            (35.0, WeatherType.SNOWY),
            (30.0, WeatherType.SUNNY),
            (25.0, WeatherType.CLOUDY),
        ],
        "Toronto": [
            (80.0, WeatherType.CLOUDY),
            (75.0, WeatherType.RAINY),
            (70.0, WeatherType.WINDY),
            (65.0, WeatherType.SNOWY),
            (60.0, WeatherType.SUNNY),
            (55.0, WeatherType.CLOUDY),
            (50.0, WeatherType.RAINY),
        ],
        "Portland": [
            (55.0, WeatherType.RAINY),
            (50.0, WeatherType.WINDY),
            (45.0, WeatherType.SNOWY),
            (40.0, WeatherType.SUNNY),
            (35.0, WeatherType.CLOUDY),
            (30.0, WeatherType.RAINY),
            (25.0, WeatherType.WINDY),
        ],
        "Austin": [
            (90.0, WeatherType.WINDY),
            (85.0, WeatherType.SNOWY),
            (80.0, WeatherType.SUNNY),
            (75.0, WeatherType.CLOUDY),
            (70.0, WeatherType.RAINY),
            (65.0, WeatherType.WINDY),
            (60.0, WeatherType.SNOWY),
        ],
        "Houston": [
            (85.0, WeatherType.SNOWY),
            (80.0, WeatherType.SUNNY),
            (75.0, WeatherType.CLOUDY),
            (70.0, WeatherType.RAINY),
            (65.0, WeatherType.WINDY),
            (60.0, WeatherType.SNOWY),
            (55.0, WeatherType.SUNNY),
        ],
        "Springfield": [
            (65.0, WeatherType.SUNNY),
            (60.0, WeatherType.CLOUDY),
            (55.0, WeatherType.RAINY),
            (50.0, WeatherType.WINDY),
            (45.0, WeatherType.SNOWY),
            (40.0, WeatherType.SUNNY),
            (35.0, WeatherType.CLOUDY),
        ],
    }
    
    if location not in location_forecast_map:
        raise LocationNotFoundException(f"Location {location} not found")
    
    return location_forecast_map[location]

def get_long_term_forecast(location: str) -> str:
    location_long_term_map = {
        "New York": "The upcoming months will bring a mix of weather. Expect a lot of rain and some sun. The temperature will be mild.",
        "Los Angeles": "The upcoming months will be hot and dry. Expect a lot of sun and very little rain. The temperature will be hot.",
        "Chicago": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Miami": "The upcoming months will be hot and wet. Expect a lot of sun and rain. The temperature will be hot.",
        "Seattle": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Orlando": "The upcoming months will be hot and wet. Expect a lot of sun and rain. The temperature will be hot.",
        "Denver": "The upcoming months will be cold and dry. Expect a lot of snow and very little rain. The temperature will be cold.",
        "San Francisco": "The upcoming months will be cold and wet. Expect a lot of rain and very little sun. The temperature will be cold.",
        "Boston": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Las Vegas": "The upcoming months will be hot and dry. Expect a lot of sun and very little rain. The temperature will be hot.",
        "Vancouver": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Toronto": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Portland": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Austin": "The upcoming months will be hot and dry. Expect a lot of sun and very little rain. The temperature will be hot.",
        "Houston": "The upcoming months will be hot and wet. Expect a lot of sun and rain. The temperature will be hot.",
        "Springfield": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
    }
    
    if location not in location_long_term_map:
        raise LocationNotFoundException(f"Location {location} not found")
    
    return location_long_term_map[location]

def available_locations() -> List[str]:
    return [
        "New York",
        "Los Angeles",
        "Chicago",
        "Miami",
        "Seattle",
        "Orlando",
        "Denver",
        "San Francisco",
        "Boston",
        "Las Vegas",
        "Vancouver",
        "Toronto",
        "Portland",
        "Austin",
        "Houston",
        "Springfield",
    ]

In [12]:
from typing import Callable
"""
The node below is no longer need since we are using the FunctionNode to recreate it
but we keep it here to show the added simplicity benefits.
"""

# class CurrentWeather(no.Node[str]):
    
#     def __init__(self, location: str, context: BaseContext):
#         super().__init__(context)
#         self.location = location
        
#     def invoke(
#         self,
#     ) -> str:
#         try:
#             temp, w_type = get_current_weather(self.location)
#             return f"The current weather in {self.location} is {temp} degrees and {w_type.value}"
#         except LocationNotFoundException as e:
#             raise no.CompletionException(self, str(e), "No weather information found")
        
#     @classmethod
#     def pretty_name(cls):
#         return "Current Weather"
        
        
class WeeklyForecast(no.Node[List[str]]):
    
    def __init__(self, location: str):
        super().__init__()
        self.location = location
        
    def invoke(
        self,
    ) -> List[str]:
        try:
            forecast = get_weekly_forecast(self.location)
            return [f"{temp} degrees and {w_type.value}" for temp, w_type in forecast]
        except LocationNotFoundException as e:
            raise no.CompletionException(self, str(e), [])
        
    @classmethod
    def pretty_name(cls):
        return "Weekly Forecast"
        
class LongTermForecast(no.Node[str]):
    
    def __init__(self, location: str):
        super().__init__()
        self.location = location
        
    def invoke(
        self,
    ) -> str:
        try:
            forecast = get_long_term_forecast(self.location)
            return forecast
        except LocationNotFoundException as e:
            raise no.CompletionException(self, str(e), "No weather information found")
    
    @classmethod
    def pretty_name(cls):
        return "Long Term Forecast"

In [13]:
from src.systems.request_completion.state.run.run import run
from src.systems.request_completion.state.context.context import EmptyContext

In [14]:
question = "Tell me about the weather in Boston in the next week. What should I expect?"  
mess_hist = llm.MessageHistory([llm.UserMessage(question)])

complete = run(
    start_node=WeatherForecaster(mess_hist,),
)

print(complete.answer)



Here is the weather forecast for Boston in the next week:

- **Day 1:** 65°F and windy
- **Day 2:** 60°F and snowy
- **Day 3:** 55°F and sunny
- **Day 4:** 50°F and cloudy
- **Day 5:** 45°F and rainy
- **Day 6:** 40°F and windy
- **Day 7:** 35°F and snowy

Be prepared for a mix of weather conditions, including wind, snow, rain, and some sunny days.
